In [32]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [33]:
TRAIN_PATH = "./data/spam_data_train.rda"
TEST_PATH = "./data/spam_data_test.rda"

In [34]:
ro.r["load"](TRAIN_PATH)
ro.r["load"](TEST_PATH)

array(['data_test'], dtype='<U9')

In [ ]:
train_data = ro.r["data_train"]
test_data = ro.r["data_test"]

train_df = pandas2ri.rpy2py(train_data)
test_df = pandas2ri.rpy2py(test_data)
print(train_data.head())
print(test_data.head())


#############################################
     word_freq_make word_freq_address word_freq_all word_freq_3d word_freq_our
2961           0.00              0.00          0.00         0.00          0.00
2515           0.00              0.00          0.00         0.00          0.00
3246           0.00              0.00          0.00         0.00          0.00
1117           0.58              0.00          0.00        35.46          0.58
2340           0.69              0.00          0.69         0.00          0.00
1593           0.77              0.38          0.77         0.00          0.00
     word_freq_over word_freq_remove word_freq_internet word_freq_order
2961           0.00             0.00               0.00            0.00
2515           0.00             0.00               0.00            0.00
3246           0.00             0.00               0.00            0.00
1117           0.00             0.58               0.58            0.00
2340           0.00             0.00     

## 2. ACP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
Z = train_df - np.mean(train_df)
S = np.cov(Z, rowvar=False)
ls, vs = np.linalg.eig(S)
idx = np.argsort(ls)[::-1]
ls_tries = ls[idx]
vs_tries = vs[:, idx]
print(f'pourcentage de variance gardée : {ls_tries[0]/np.sum(ls_tries)}')
Z_new = np.dot(vs_tries[0], Z.T)
plt.grid()
plt.scatter(Z[:,0], Z[:,1])
plt.savefig("ACP.png")

NameError: name 'train_df' is not defined

## 3. Entraînement des modèles

In [39]:
# Séparation des caractéristiques et des étiquettes
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']
X_test = test_data.drop('label', axis=1)
y_test = test_data['label']


AttributeError: 'DataFrame' object has no attribute 'drop'

### 3.1 Naive Bayes

In [38]:
from Naive_Bayes import NaiveBayes

# Instancier et entraîner le classificateur Naive Bayes
nb_classifier = NaiveBayes()
nb_classifier.fit(X_train, y_train)

# Prédire les étiquettes sur l'ensemble de test
nb_predictions = nb_classifier.predict(X_test)

# Calculer l'exactitude
nb_accuracy = nb_classifier.accuracy(y_test, nb_predictions)
print(f"L'exactitude du modèle est de {nb_accuracy * 100:.2f}%")

NameError: name 'X_train' is not defined

### 3.2 Régression logistique

In [ ]:
from LogisticRegression import LogisticRegression

# Instancier et entraîner le classificateur de régression logistique
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train.values, y_train.values)

# Prédire les étiquettes sur l'ensemble de test
lr_predictions = lr_classifier.predict(X_test.values)

# Calculer l'exactitude
lr_accuracy = lr_classifier.accuracy(y_test.values, lr_predictions)
print(f"L'exactitude du modèle est de {lr_accuracy * 100:.2f}%")